In [ ]:
# This is a Python 3 environment

#Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error 

import os

print(os.listdir("../input"))


# Motivation for Project
I am interested in this project because I have a few friends who have or currently rent their spots out through AirBnb. I would like to be able to present them my findings so they will better be able to maximize their rental business.  


https://github.com/pranjalvj05/Host_activity_Seattle_AirBNB_Data
Question 1: What are the main accomodations / features that have affected reservation price the most?

that, if the size of room, host experiences, location etc. most affect the reservation price rates. So that we can keep that in mind before reserved accomodation.


Question 2: Is there any seasonality in the price?
that is , Does time of the year influence prices? and what are the busiest times of the year to visit particular city? In which season that is summer , winter , spring according to their month are the price high there.



https://github.com/paulstancliffe/Blog-Post-AirBnB-Seattle/blob/master/seattle_airbnb-v2.ipynb
1) Business Understanding
what are the questions you are interested in tackling.
a) What is the monthly average price/night for the city?
b) What is the average price Monday to Sunday?
c) What is the monthly occupancy ratio for the city?
d) What is the average price/night per neighbourhood?
e) What is the occupancy ratio per neighbourhood?